We're going to explore running agents with Amazon Bedrock.  

In [68]:
from langchain.tools import tool
from langchain.agents import AgentType, initialize_agent
from langchain.llms.bedrock import Bedrock
from langchain import hub
import boto3


This requires you to have a named profile configured in your ~/.aws/config or ~/.aws/credentials file.  This supports AWS IC if you have that configured with your CLI.  Just pass it the profile name you use after successfully authenticating with 

```bash
aws sso login --profile <profile name>
```
Note: this can be run from anywhere on your computer not necessarily the same terminal session.

Enter the name of the profile line 2 below.

In [ ]:
REGION_NAME="us-east-1" ## change to your region
PROFILE_NAME="lza-comm-gss"  ## change to your desired aws credential profile
## ensure AI21 Jurassic Ultra is enabled in your AWS Account.
named_profile = boto3.session.Session(profile_name=PROFILE_NAME)
bedrock_client = named_profile.client('bedrock-runtime')
print('Initalizing Bedrock AI21 Jurassic Ultra')
llm = Bedrock(
    client=bedrock_client,
    model_id="ai21.j2-ultra-v1",
    endpoint_url="https://bedrock-runtime." + REGION_NAME + ".amazonaws.com",
    model_kwargs={"temperature": 0.2, "maxTokens": 1200, "numResults": 1}
)

In [ ]:
llm.invoke("What is the product of 4 and 28?")

The above command may stumble upon the correct answer 112 - but running it several times will show that it is rather inconsistent.  LLMs do not perform well with Math on their own. 

What we need to do is build in logic for the LLM to multiply two numbers together.  This is where tools come in...  

We can define them with a decorator function.

And then we add them to an array tools, where we can store multiple tools for our use to pass to our agent.

In [ ]:
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers. Find the product of two numbers a and b."""
    return a * b

tools=[multiply]

We initialize the agent and bind it to with the tools and our bedrock FM.  

Notice that we use the STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION AgentType - this is because our tool requires multiple arguments - the two numbers to multiply.

We pass the verbose argument so we can see a clear Chain of Thought.


In [ ]:
base_prompt = hub.pull("langchain-ai/react-agent-template")

agent = initialize_agent(
    tools, llm, agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True, prompt=base_prompt
)

Invoke the model...

In [ ]:
agent.run("What is the product of 94 and 28?")


In [132]:
@tool
def fibonacci(n: int) -> int:
    """Calculate the nth number in the Fibonacci sequence."""
    a, b = 0, 1
    for _ in range(n):
        a, b = b, a+b
    return a


tools=[fibonacci]

In [130]:
#fibonacci(110)

In [133]:
from langchain import hub
from langchain_experimental.tools import PythonREPLTool
from langchain.schema import StrOutputParser

llm = Bedrock(
    client=bedrock_client,
    model_id="anthropic.claude-v2",
    endpoint_url="https://bedrock-runtime." + REGION_NAME + ".amazonaws.com",
    model_kwargs={"temperature": 0}
)
# instructions = """You are an agent designed to write and execute python code to answer questions.
# You have access to a python REPL, which you can use to execute python code.
# If you get an error, debug your code and try again.
# Only use the output of your code to answer the question. 
# You might know the answer without running any code, but you should still run the code to get the answer.
# If it does not seem like you can write code to answer the question, just return "I don't know" as the answer.
# """
base_prompt = hub.pull("langchain-ai/react-agent-template")
#print(base_prompt)
#prompt = base_prompt.partial(instructions=instructions)
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, prompt=base_prompt
) #| StrOutputParser()

agent.invoke({"input": "What is the 12th fibonacci number?"})



> Entering new AgentExecutor chain...
 Here is my attempt to answer the question using the provided format:

Question: What is the 12th fibonacci number?

Thought: To find the 12th Fibonacci number, I need to call the fibonacci function with input 12.

Action: fibonacci 
Action Input: 12

TypeError: 'str' object cannot be interpreted as an integer

# BOOM!

### Further reading:

[Zero Shot vs Few Shot Prompting](https://machinelearningmastery.com/what-are-zero-shot-prompting-and-few-shot-prompting/)

[ReAct: Reasoning + Action](https://arxiv.org/abs/2210.03629)